In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

from sklearn.preprocessing import KBinsDiscretizer, OneHotEncoder, MinMaxScaler
from sklearn.metrics.pairwise import haversine_distances
import category_encoders as ce

# Preprocess the dataset

In [2]:
######### Read the data #########
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')

######### Remove duplicates and irrelevant features #########
train_df.drop_duplicates(inplace=True)
train_df.drop(columns=['furnished','elevation','block','town', 'street_name', 'subzone'], inplace=True)
test_df.drop(columns=['furnished','elevation','block','town', 'street_name', 'subzone'], inplace=True)

######### One hot encoding for some of categorical variables #########
ohe = ce.OneHotEncoder(use_cat_names=True)
ohe_features = ['planning_area','region','flat_model']

train_df = pd.concat([train_df, ohe.fit_transform(train_df[ohe_features]) ], axis=1).drop(ohe_features, axis=1)
test_df = pd.concat([test_df, ohe.transform(test_df[ohe_features])], axis=1).drop(ohe_features, axis=1)

######### Ordinal Encoding #########
ordinal_cols_mapping = [{'col': 'flat_type', 'mapping': {'2 room': 2, '3 room': 3, '4 room': 4, '5 room': 5, 'executive': 6}},]
flat_type_encoder = ce.OrdinalEncoder(mapping = ordinal_cols_mapping, return_df = True) 

train_df = pd.concat([train_df, flat_type_encoder.fit_transform(train_df['flat_type']) ], axis=1).drop('flat_type', axis=1)
test_df = pd.concat([test_df, flat_type_encoder.transform(test_df['flat_type'])], axis=1).drop('flat_type', axis=1)

######### set target variables #########
train_y = train_df['monthly_rent']
train_df = train_df.loc[:,train_df.columns != 'monthly_rent']

######### Convert latitude and longitude to rads #########
train_df['lat_rads'] = np.radians(train_df['latitude'])
train_df['long_rads'] = np.radians(train_df['longitude'])
test_df['lat_rads'] = np.radians(test_df['latitude'])
test_df['long_rads'] = np.radians(test_df['longitude'])

train_df.drop(['latitude', 'longitude'], axis = 1, inplace = True)
test_df.drop(['latitude', 'longitude'], axis = 1, inplace = True)

In [3]:
test_df.shape

(30000, 58)

# Add stock market index value

In [4]:
# Get STI index values from start date to end date
train_df['rent_approval_date'] = pd.to_datetime(train_df['rent_approval_date'])
train_df['rent_approval_date'].sort_values()
test_df['rent_approval_date'] = pd.to_datetime(test_df['rent_approval_date'])
test_df['rent_approval_date'].sort_values()
start_date = min(train_df['rent_approval_date'].min(), test_df['rent_approval_date'].min()) - pd.DateOffset(days=1)
end_date = max(train_df['rent_approval_date'].max(), test_df['rent_approval_date'].max())
yf_df = yf.download('^STI', start=start_date, end=end_date)

sti_df = pd.DataFrame(index=pd.date_range(start_date, end_date, freq='d'))
sti_df = pd.merge(sti_df, yf_df[['Adj Close']], how='left', left_index=True, right_index=True)
sti_df.fillna(method='ffill', inplace=True)
sti_df.columns = ['sti']

# add sti values to train and test data
scaler = MinMaxScaler()
sti_df.loc[:,'sti'] = scaler.fit_transform(sti_df['sti'].values.reshape(-1, 1))
# find closest date to rent_approval_date
train_df['rent_approval_date'] = pd.to_datetime(train_df['rent_approval_date'])
test_df['rent_approval_date'] = pd.to_datetime(test_df['rent_approval_date'])
train_df['rent_approval_date'] = train_df['rent_approval_date'].dt.date
test_df['rent_approval_date'] = test_df['rent_approval_date'].dt.date
sti_df.index = sti_df.index.date
train_df['sti'] = train_df['rent_approval_date'].map(sti_df['sti'])
test_df['sti'] = test_df['rent_approval_date'].map(sti_df['sti'])

[*********************100%***********************]  1 of 1 completed


# COE Prices

In [5]:
coe_df = pd.read_csv('./data/auxiliary-data/sg-coe-prices.csv')

coe_df['date'] = (pd.to_datetime(coe_df['year'].astype(str)  + coe_df['month'], format='%Y%B'))
coe_df.loc[coe_df['bidding']==2, 'date'] += pd.DateOffset(days=15)

coe_df['date'] = (pd.to_datetime(coe_df['year'].astype(str)  + coe_df['month'], format='%Y%B'))
print(coe_df['category'].unique())
coe_cal_date_df = pd.DataFrame({'price': np.nan}, index=pd.date_range(start_date, end_date, freq='d'))
for date in coe_df.loc[coe_df['category'] == 'e', 'date'].unique():
    coe_cal_date_df.loc[date, 'price'] = np.mean(coe_df.loc[(coe_df['category'] == 'e') & (coe_df['date'] == date), 'price'])
coe_cal_date_df.fillna(method='ffill', inplace=True)
coe_cal_date_df.columns = ['coe']
coe_cal_date_df.dropna(inplace=True)

# add coe values to train and test data
scaler = MinMaxScaler()
coe_cal_date_df.loc[:,'coe'] = scaler.fit_transform(coe_cal_date_df['coe'].values.reshape(-1, 1))
# find closest date to rent_approval_date
train_df['rent_approval_date'] = pd.to_datetime(train_df['rent_approval_date'])
test_df['rent_approval_date'] = pd.to_datetime(test_df['rent_approval_date'])
train_df['rent_approval_date'] = train_df['rent_approval_date'].dt.date
test_df['rent_approval_date'] = test_df['rent_approval_date'].dt.date
coe_cal_date_df.index = coe_cal_date_df.index.date
train_df['coe'] = train_df['rent_approval_date'].map(coe_cal_date_df['coe'])
test_df['coe'] = test_df['rent_approval_date'].map(coe_cal_date_df['coe'])

['a' 'b' 'c' 'e']


# Age

In [6]:
train_df['rent_approval_date'] = train_df['rent_approval_date'].astype("datetime64[ns]")
train_df["lease_commence_date"] = pd.to_datetime(train_df["lease_commence_date"].astype(str), format='%Y')#.astype('datetime64[ns]')
train_df["age"] = (train_df["rent_approval_date"] - train_df["lease_commence_date"])/np.timedelta64(1, 'Y')

test_df['rent_approval_date'] = test_df['rent_approval_date'].astype("datetime64[ns]")
test_df["lease_commence_date"] = pd.to_datetime(test_df["lease_commence_date"].astype(str), format='%Y')#.astype('datetime64[ns]')
test_df["age"] = (test_df["rent_approval_date"] - test_df["lease_commence_date"])/np.timedelta64(1, 'Y')

train_df.drop(['rent_approval_date', 'lease_commence_date'], axis=1, inplace=True)
test_df.drop(['rent_approval_date', 'lease_commence_date'], axis=1, inplace=True)

# Proximity to malls

In [7]:
#Proximity to malls
mall_df = pd.read_csv('./data/auxiliary-data/sg-shopping-malls.csv')
mall_df['lat_rads'] = np.radians(mall_df['latitude'])
mall_df['long_rads'] = np.radians(mall_df['longitude'])
distance = haversine_distances(train_df[['lat_rads', 'long_rads']], mall_df[['lat_rads', 'long_rads']])
distance = distance * 6371000/1000  # multiply by Earth radius to get kilometers
train_df['shortest_dist_mall'] = distance.min(axis=1)
distance = haversine_distances(test_df[['lat_rads', 'long_rads']], mall_df[['lat_rads', 'long_rads']])
distance = distance * 6371000/1000  # multiply by Earth radius to get kilometers
test_df['shortest_dist_mall'] = distance.min(axis=1)

#Proximity to MRT Stations
existing_mrt_df = pd.read_csv('./data/auxiliary-data/sg-mrt-existing-stations.csv')
existing_mrt_df['lat_rads'] = np.radians(existing_mrt_df['latitude'])
existing_mrt_df['long_rads'] = np.radians(existing_mrt_df['longitude'])
distance = haversine_distances(train_df[['lat_rads', 'long_rads']], existing_mrt_df[['lat_rads', 'long_rads']])
distance = distance * 6371000/1000  # multiply by Earth radius to get kilometers
train_df['shortest_ex_mrt'] = distance.min(axis=1)
distance = haversine_distances(test_df[['lat_rads', 'long_rads']], existing_mrt_df[['lat_rads', 'long_rads']])
distance = distance * 6371000/1000  # multiply by Earth radius to get kilometers
test_df['shortest_ex_mrt'] = distance.min(axis=1)

#Proximity to MRT Stations
planned_mrt_df = pd.read_csv('./data/auxiliary-data/sg-mrt-existing-stations.csv')
planned_mrt_df['lat_rads'] = np.radians(planned_mrt_df['latitude'])
planned_mrt_df['long_rads'] = np.radians(planned_mrt_df['longitude'])
distance = haversine_distances(train_df[['lat_rads', 'long_rads']], planned_mrt_df[['lat_rads', 'long_rads']])
distance = distance * 6371000/1000  # multiply by Earth radius to get kilometers
train_df['shortest_pl_mrt'] = distance.min(axis=1)
distance = haversine_distances(test_df[['lat_rads', 'long_rads']], planned_mrt_df[['lat_rads', 'long_rads']])
distance = distance * 6371000/1000  # multiply by Earth radius to get kilometers
test_df['shortest_pl_mrt'] = distance.min(axis=1)

# Check if there is still any categorical features

In [8]:
train_df.dtypes[train_df.dtypes != 'int64'][train_df.dtypes != 'float64']

Series([], dtype: object)

In [9]:
test_df.dtypes[test_df.dtypes != 'int64'][test_df.dtypes != 'float64']

Series([], dtype: object)

# To csv

In [10]:
train_df['monthly_rent'] = train_y

In [11]:
test_df.shape

(30000, 62)

In [12]:
train_df.shape

(59727, 63)

In [13]:
train_df.head()

,floor_area_sqm,planning_area_jurong east,planning_area_bedok,planning_area_toa payoh,planning_area_pasir ris,planning_area_kallang,planning_area_bukit panjang,planning_area_sengkang,planning_area_ang mo kio,planning_area_bishan,...,flat_model_3gen,lat_rads,long_rads,sti,coe,age,shortest_dist_mall,shortest_ex_mrt,shortest_pl_mrt,monthly_rent
0,67.0,1,0,0,0,0,0,0,0,0,...,0,0.023466,1.810581,0.405905,0.258731,38.667461,1.202674,0.699127,0.699127,1600
1,92.0,0,1,0,0,0,0,0,0,0,...,0,0.023216,1.814073,0.853443,0.597983,44.329452,1.114338,0.898991,0.898991,2250
2,67.0,0,0,1,0,0,0,0,0,0,...,0,0.023252,1.812448,0.476430,0.766542,51.749180,0.468297,0.218603,0.218603,1900
3,149.0,0,0,0,1,0,0,0,0,0,...,0,0.023915,1.814495,0.537475,0.192568,28.581011,0.402359,1.546040,1.546040,2850
4,68.0,0,0,0,0,1,0,0,0,0,...,0,0.023047,1.812757,0.476863,0.880567,50.834719,1.073354,0.187856,0.187856,2100


In [14]:
test_df.head()

,floor_area_sqm,planning_area_jurong east,planning_area_bedok,planning_area_toa payoh,planning_area_pasir ris,planning_area_kallang,planning_area_bukit panjang,planning_area_sengkang,planning_area_ang mo kio,planning_area_bishan,...,flat_model_terrace,flat_model_3gen,lat_rads,long_rads,sti,coe,age,shortest_dist_mall,shortest_ex_mrt,shortest_pl_mrt
0,121.0,0,0,0,0,0,0,0,0,0,...,0,0,0.023709,1.813253,0.677828,0.778767,39.001485,0.739403,0.820333,0.820333
1,100.0,0,0,0,0,0,0,0,0,0,...,0,0,0.025243,1.812015,0.632518,0.815156,23.666468,0.254920,0.307784,0.307784
2,91.0,0,0,0,0,0,0,0,0,0,...,0,0,0.022789,1.810991,0.602295,0.961789,43.497129,0.497670,1.097689,1.097689
3,74.0,1,0,0,0,0,0,0,0,0,...,0,0,0.023472,1.810444,0.537475,0.192568,35.581839,1.596397,0.362810,0.362810
4,121.0,1,0,0,0,0,0,0,0,0,...,0,0,0.023482,1.810521,0.723253,0.668588,39.163022,1.461487,0.448980,0.448980


In [15]:
train_df.to_csv('./data/train_svm.csv', index=False)
test_df.to_csv('./data/test_svm.csv', index=False)

In [16]:
test_df.shape, train_df.shape

((30000, 62), (59727, 63))